Let's start to convert league of legends match data in json to sequence database
For each match, get champion name of each participants
[info][participants][championName]

In [1]:
import json
import os

MATCH_PATH = './match'

# files paths: ./match
sequences = []
# open and read every files of the directory match
for file in os.listdir(MATCH_PATH):
    with open(MATCH_PATH + '/' + file, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except:
            continue
    
    champion_team_a = []
    champion_team_b = []
    # extract the champion played by the players
    info = data.get('info')
    if info:
        participants = info.get('participants')
        if participants:
            for player in participants:
                teamId = player.get('teamId')
                championName = player.get('championName')
                if teamId and teamId == 100:
                    champion_team_a.append(championName)
                elif teamId and teamId == 200:
                    champion_team_b.append(championName)
    
    # add the sequence to the list
    sequences.append(champion_team_a)
    sequences.append(champion_team_b)

# print
# print(sequences)
    

Creates differents sequence database for duo champions

In [6]:
# divide each sequences in sub-sequences of each duo of champions
# ex: [Aatrox, Ahri, Akali] -> [[Aatrox, Ahri], [Aatrox, Akali], [Ahri, Akali]]

# list of all the sub-sequences
sub_sequences = []
for sequence in sequences:
    # list of the sub-sequences of the current sequence
    sub_sequence = []
    for i in range(len(sequence)):
        for j in range(i+1, len(sequence)):
            sub_sequence.append([sequence[i], sequence[j]])
    sub_sequences += sub_sequence

# print
# print(sub_sequences)

Then use the SPADE algorithm to mine for frequent sequential patterns. The SPADE algorithm works by finding frequent itemsets (sets of items that appear together frequently) and then extending these itemsets into longer sequential patterns. You can specify a minimum support threshold, which determines the minimum frequency at which a pattern will be considered frequent.

In [24]:
# Use the spade algorithm to find the frequent sequences
# do the spade algorythm from scratch without using the library

def spade(sub_sequences, min_support):
    # count the number of occurence of each sub-sequences
    count = {}
    for sub_sequence in sub_sequences:
        key = str(sub_sequence)
        if key in count:
            count[key] += 1
        else:
            count[key] = 1

    # print
    # print(count)

    # keep only the sub-sequences that have a support greater than the min_support
    frequent_sequences = []
    for sub_sequence in count:
        if count[sub_sequence] >= min_support:
            frequent_sequences.append(sub_sequence)

    # print
    # print(frequent_sequences)

    return frequent_sequences

# print
print(spade(sub_sequences, 30))

threshold = 0
while spade(sub_sequences, threshold) != []:
    threshold += 1
print('Limit threshold: ' + str(threshold))

["['Lucian', 'Nami']", "['Kaisa', 'Nautilus']"]
Limit threshold: 87
